# Predicting the Past, United States Extension (Decay on) 

## Introduction
This is an add on of Predicting the Past, United States Extension where decay is now turned on. 
This add on includes comparisons between _CYCLUS_ output and CURIE data for the United States from 1968 to 2013 where decay is turned on for: 
 - Total spent fuel mass 
 - Mass of major isotopes in spent fuel 

The CURIE data is taken from UNF_ST&DARDS Unified Database and the Automatic Document Generator. 

The _CYCLUS_ data was generated by using published data of the commericial reactors that have operated in the United States. The _CYCLUS_ input file and simulation were generated in the original Predicting the Past, United States notebook. This notebook uses the SQL data file produced from the _CYCLUS_ simulation to do further analysis. 

Decay was turned on by adding 10 storage facilities to the input xml file that each take in consecutive ~ 5 years of spent fuel (with exception to 1st storage facility that takes ~ 10 years and 10th that takes everything after 2014) and outputs the spent fuel at year 2020. 

Most of the subsequent code is copied from united_states_extention.ipynb

** Import necessary libraries ** 

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import csv
import collections
import dateutil.parser as date
import matplotlib.pyplot as plt
import numpy as np
import os
import pathlib
import sys
import operator
import re 
import sqlite3 as lite
from matplotlib import cm
from pyne import nucname as nn
from IPython.display import Image, HTML, display
import seaborn as sns 

sys.path.append('../../../scripts/')
import analysis as an

** Run Cyclus simulation **

In [ ]:
!rm cyclus/united_states_decayon_underestm_33.sqlite
!cyclus -i ./cyclus/input/united_states_underestm_33.xml -o ./cyclus/united_states_decayon_underestm_33.sqlite --warn-limit 0

** Setting up to analyze sqlite file ** 

In [8]:
# Before this step, must download the output sqlite files from the fuel-cycles box. 
# fuel-cycle/cyclus_output/predicting_the_past_validation 
# and put into a directory named cyclus 

def get_cursor(file_name):
    """ Connects and returns a cursor to an sqlite output file

    Parameters
    ----------
    file_name: str
        name of the sqlite file

    Returns
    -------
    sqlite cursor
    """
    con = lite.connect(file_name)
    con.row_factory = lite.Row
    return con.cursor()


cursor_51 = get_cursor('cyclus/united_states_decayon_underestm.sqlite')
cursor_33 = get_cursor('cyclus/united_states_decayon_underestm_33.sqlite')

In [9]:
def get_timesteps(cur):
    """ Returns simulation start year, month, duration and
    timesteps (in numpy linspace).

    Parameters
    ----------
    cur: sqlite cursor
        sqlite cursor

    Returns
    -------
    init_year: int
        start year of simulation
    init_month: int
        start month of simulation
    duration: int
        duration of simulation
    timestep: list
        linspace up to duration
    """
    info = cur.execute('SELECT initialyear, initialmonth, '
                       'duration FROM info').fetchone()
    init_year = info['initialyear']
    init_month = info['initialmonth']
    duration = info['duration']
    timestep = np.linspace(0, duration - 1, num=duration)

    return init_year, init_month, duration, timestep

** Get total spent fuel produced after cooling in spent fuel pool until 2020** 

In [11]:
def total_spent_fuel_produced(cur): 
    """ Find timeseries of mass of spent fuel output by reactors 
    
    Parameters 
    ----------
    cur: sqlite cursor 
    
    Returns 
    -------
    timeseries list of spent fuel output by reactors
    """
    coollist = ["cool_spent_uox1","cool_spent_uox2","cool_spent_uox3","cool_spent_uox4","cool_spent_uox5","cool_spent_uox6","cool_spent_uox7","cool_spent_uox8","cool_spent_uox9"]
    total_list = []
    
    for x in range(0,9):
        cool_spent_uox = coollist[x]
        
        init_yr, init_month, duration, timestep = get_timesteps(cur)
        spent_fuel = cur.execute('SELECT time, sum(quantity) FROM transactions '
                                 'INNER JOIN resources '
                                 'ON transactions.resourceid = resources.resourceid '
                                 'WHERE Commodity =:cool_spent_uox' # must specify isotope and pool
                                 ' GROUP BY time ',{"cool_spent_uox": cool_spent_uox}).fetchall()
        spent_fuel_list = an.get_timeseries(spent_fuel,duration,True)
        spent_fuel_total = np.sum(spent_fuel_list)
        total_list.append(spent_fuel_total)
        
    total_cum = np.cumsum(total_list)
    return total_cum 

total_spentfuel_33 = total_spent_fuel_produced(cursor_33)
total_spentfuel_51 = total_spent_fuel_produced(cursor_51)

** Get specific isotope composition from total spent fuel produced after cooling in spent fuel pool until 2020** 

In [ ]:
def isotope_total_cum(cur,num,in_dict):
    """ Find total isotopes present in spent fuel output by reactor for each cooling pool and sum cumulatively 
    
    Parameters 
    ----------
    cur: sqlite cursor 
    num: nucid 
    
    Returns 
    -------
    list of isotopes in spent fuel output by reactors added cumulatively for each cooling pool
    """    
    
    coollist = ["cool_spent_uox1","cool_spent_uox2","cool_spent_uox3","cool_spent_uox4","cool_spent_uox5","cool_spent_uox6","cool_spent_uox7","cool_spent_uox8","cool_spent_uox9"]
    isotope_total_list = []
    
    for x in range(0,9):
        cool_spent_uox = coollist[x]
    
        init_yr, init_month, duration, timestep = get_timesteps(cur)
        isotopes = cur.execute('SELECT time, sum(quantity)*massfrac FROM transactions '
                               'INNER JOIN resources '
                               'ON transactions.resourceid = resources.resourceid '
                               'LEFT OUTER JOIN compositions '
                               'ON resources.qualid = compositions.qualid '
                               'WHERE Commodity =:cool_spent_uox AND nucid =:num ' # must specify isotope and pool
                               ' GROUP BY time ',{"cool_spent_uox": cool_spent_uox, "num":num}).fetchall()
        isotope_list = an.get_timeseries(isotopes,duration,True)
        isotope_total = np.sum(isotope_list)
        isotope_total_list.append(isotope_total)
    
    isotope_total_cum = np.cumsum(isotope_total_list)
    #print(isotope_total_cum)
    name = nn.name(num)
    #print(name)
    in_dict[name] = isotope_total_cum
    #print(dict_51)
    return

In [12]:
dict_51 = {}
nucnum = [20040000, 882260000, 882280000, 822060000, 822070000, 822080000, 822100000, 902280000, 902290000, 902300000, 902320000, 832090000, 892270000, 912310000, 922320000, 922330000, 922340000, 922350000, 922360000, 922380000, 932370000, 942380000, 942390000, 942400000, 942410000, 942420000, 942440000, 952410000, 952420001, 952430000, 962420000, 962430000, 962440000, 962450000, 962460000, 962470000, 962480000, 962500000, 982490000, 982500000, 982510000, 982520000, 10030000, 60140000, 360810000, 360850000, 380900000, 430990000, 531290000, 551340000, 551350000, 551370000]
for x in range(0,52):
    num = nucnum[x]
    print(x)
    isotope_total_cum(cursor_51,num,dict_51)

0


NameError: name 'isotope_total_cum' is not defined

In [ ]:
dict_33 = {}
for x in range(0,52):
    num = nucnum[x]
    print(x)
    isotope_total_cum(cursor_33,num,dict_33)

In [ ]:
%store dict_51

In [ ]:
%store dict_33

In [13]:
%store -r dict_51

In [14]:
%store -r dict_33

In [19]:
try:
    import cPickle as pickle 
except ImportError: 
    import pickle 
    
with open('dict_51.p','wb') as fp:
    pickle.dump(dict_33,fp,protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
try:
    import cPickle as pickle 
except ImportError: 
    import pickle 
    
with open('dict_33.p','wb') as fp:
    pickle.dump(dict_33,fp,protocol=pickle.HIGHEST_PROTOCOL)

## CURIE Data Analysis 

In [ ]:
names = ['assembly_id','reactor_id', 'reactor_type', 'initial_uranium_kg','initial_enrichment','discharge_burnup','discharge_date','discharge_time','total_assembly_decay_heat_kw','name','evaluation_date','total_mass_g','total_radioactivity_curies']
df = pd.read_csv('SNF_nuclide_2020_07_01.dat',
                  sep='\s+',
                  index_col=False, 
                  names = names)

In [ ]:
date_isotope_mass = df[['discharge_date','name','total_mass_g']]
date_mass = df[['discharge_date','total_mass_g']]

In [ ]:
spent_fuel_mass = date_mass.groupby('discharge_date').sum()
spent_fuel_mass_cum = spent_fuel_mass.cumsum()
spent_fuel_mass_cum['total_mass_g'] = spent_fuel_mass_cum['total_mass_g'].apply(lambda x: x*0.000001)
spent_fuel_mass_cum = spent_fuel_mass_cum.rename(columns = {'total_mass_g':'total_mass_MTHM_CURIE'})
spent_fuel_mass_cum.index.names = ['discharge_date']

In [ ]:
CURIE_isotope = date_isotope_mass.pivot_table(index='discharge_date',columns='name',aggfunc=sum)
CURIE_isotope_cum_all = CURIE_isotope.cumsum()

In [ ]:
pd.set_option('display.max_columns',None)
isotope_list = list(CURIE_isotope_cum_all)
curie_isotope_list = [x[1] for x in isotope_list]

## Relative Error 

In [ ]:
def relative_error_total(cyclus_list,pool): 
    """ Find relative error between the total spent fuel output in 2014 for cyclus and udb data 
    
    Parameters 
    ----------
    cyclus_list: array of cumulative spent fuel mass 
    
    Returns 
    -------
    relative errors in percentage of spent fuel mass in 2014
    """   

    curie_list = spent_fuel_mass_cum.values
    cyclus_2014 = cyclus_list[pool-1] 
    curie = [0, 0, 0, 0, 0, 1167, 0, 0, -1]
    curie_2014 = curie_list[curie[pool-1]]
    
    relative_error= ((cyclus_2014-curie_2014)/curie_2014)*100
    print(relative_error)
    
    return relative_error     

In [ ]:
total_relative_error_9=relative_error_total(total_spentfuel_33,9)
total_relative_error_6=relative_error_total(total_spentfuel_33,6)

In [ ]:
def isotope_error(in_dict,pool,absolute): 
    """ Find relative error between the total isotopes present in spent fuel output in 2014 for cyclus and udb data 
    
    Parameters 
    ----------
    in_dict = dictionary of isotopes and their corresponding cumilative spent fuel mass 
    pool = int of pool number you want to compare isotopes at 
    
    Returns 
    -------
    relative errors in percentage of the specified isotope for burn up of 51 and 33 GWd/tHM in 2014 in a dictionary 
    """    
    isotope_names = ['He4', 'Ra226', 'Ra228', 'Pb206', 'Pb207', 'Pb208', 'Pb210', 'Th228', 'Th229', 'Th230', 'Th232', 'Bi209', 'Ac227', 'Pa231', 'U232', 'U233', 'U234', 'U235', 'U236', 'U238', 'Np237', 'Pu238', 'Pu239', 'Pu240', 'Pu241', 'Pu242', 'Pu244', 'Am241', 'Am242M', 'Am243', 'Cm242', 'Cm243', 'Cm244', 'Cm245', 'Cm246', 'Cm247', 'Cm248', 'Cm250', 'Cf249', 'Cf250', 'Cf251', 'Cf252', 'H3', 'C14', 'Kr81', 'Kr85', 'Sr90', 'Tc99', 'I129', 'Cs134', 'Cs135', 'Cs137']
    error_isotope = {}
    for x in range(0,52): 
        cyclus_2014 = in_dict[isotope_names[x]][pool-1]
        curie = [0, 0, 0, 0, 0, 1167, 0, 0, -1]
        #print(cyclus_2014)
        element_name = isotope_names[x]
        element_name = element_name.lower()
        match = re.match(r"([a-z]+)([0-9]+)", element_name, re.I)
        if match:
            items = match.groups()
            #print(items)
        num_name = items[0]+ "-" + items[1]
        #print(num_name)
        if any (num_name in s for s in curie_isotope_list):
            if num_name == 'am-242':
                curie_list = CURIE_isotope_cum_all.loc[:,('total_mass_g', 'am-242m')]
                curie_list = curie_list.to_frame()
                curie_list = curie_list.multiply(0.000001)
                curie_list = curie_list.values

                curie_2014 = curie_list[curie[pool-1]]
                if absolute: 
                    error = (cyclus_2014-curie_2014)
                else: 
                    error = ((cyclus_2014-curie_2014)/curie_2014)*100
                error_isotope[element_name] = error  
            else: 
                curie_list = CURIE_isotope_cum_all.loc[:,('total_mass_g', num_name)]
                curie_list = curie_list.to_frame()
                curie_list = curie_list.multiply(0.000001)
                curie_list = curie_list.values

                curie_2014 = curie_list[curie[pool-1]]
                if absolute: 
                    error = (cyclus_2014-curie_2014)
                else: 
                    error = ((cyclus_2014-curie_2014)/curie_2014)*100
                error_isotope[element_name] = error 
        #else: 
            #print(num_name,'does not exist in curie data')
    return error_isotope  

In [ ]:
def relative_error_isotope(in_dict,pool): 
    """ Find relative error between the total isotopes present in spent fuel output in 2014 for cyclus and udb data 
    
    Parameters 
    ----------
    in_dict = dictionary of isotopes and their corresponding cumilative spent fuel mass 
    pool = int of pool number you want to compare isotopes at 
    
    Returns 
    -------
    relative errors in percentage of the specified isotope for burn up of 51 and 33 GWd/tHM in 2014 in a dictionary 
    """    
    isotope_names = ['He4', 'Ra226', 'Ra228', 'Pb206', 'Pb207', 'Pb208', 'Pb210', 'Th228', 'Th229', 'Th230', 'Th232', 'Bi209', 'Ac227', 'Pa231', 'U232', 'U233', 'U234', 'U235', 'U236', 'U238', 'Np237', 'Pu238', 'Pu239', 'Pu240', 'Pu241', 'Pu242', 'Pu244', 'Am241', 'Am242M', 'Am243', 'Cm242', 'Cm243', 'Cm244', 'Cm245', 'Cm246', 'Cm247', 'Cm248', 'Cm250', 'Cf249', 'Cf250', 'Cf251', 'Cf252', 'H3', 'C14', 'Kr81', 'Kr85', 'Sr90', 'Tc99', 'I129', 'Cs134', 'Cs135', 'Cs137']
    relative_error = {}
    for x in range(0,52): 
        cyclus_2014 = in_dict[isotope_names[x]][pool-1]
        curie = [0, 0, 0, 0, 0, 1167, 0, 0, -1]
        #print(cyclus_2014)
        element_name = isotope_names[x]
        element_name = element_name.lower()
        match = re.match(r"([a-z]+)([0-9]+)", element_name, re.I)
        if match:
            items = match.groups()
            #print(items)
        num_name = items[0]+ "-" + items[1]
        #print(num_name)
        if any (num_name in s for s in curie_isotope_list):
            if num_name == 'am-242':
                curie_list = CURIE_isotope_cum_all.loc[:,('total_mass_g', 'am-242m')]
                curie_list = curie_list.to_frame()
                curie_list = curie_list.multiply(0.000001)
                curie_list = curie_list.values

                curie_2014 = curie_list[curie[pool-1]]
                error = (cyclus_2014-curie_2014)/curie_2014
                relative_error[element_name] = error  
            else: 
                curie_list = CURIE_isotope_cum_all.loc[:,('total_mass_g', num_name)]
                curie_list = curie_list.to_frame()
                curie_list = curie_list.multiply(0.000001)
                curie_list = curie_list.values

                curie_2014 = curie_list[curie[pool-1]]
                error = (((cyclus_2014-curie_2014)/curie_2014))*100
                relative_error[element_name] = error 
        #else: 
            #print(num_name,'does not exist in curie data')
    return relative_error  

In [ ]:
relative_error_isotope_51_6 = relative_error_isotope(dict_51,6)
relative_error_isotope_51_9 = relative_error_isotope(dict_51,9)

relative_error_isotope_33_6 = relative_error_isotope(dict_33,6)
relative_error_isotope_33_9 = relative_error_isotope(dict_33,9)

In [ ]:
relative_error_isotope_51_6

In [ ]:
relative_error_isotope_51_6 = isotope_error(dict_51,6,True)
relative_error_isotope_51_9 = isotope_error(dict_51,9,False)

relative_error_isotope_33_6 = isotope_error(dict_33,6,False)
relative_error_isotope_33_9 = isotope_error(dict_33,9,False)

In [ ]:
relative_error_isotope_51_6

In [ ]:
def sort_relative_error(in_list_sort,in_list_copy):
    """ Sort in_list_copy based on the first column in_list_sort
    
    Parameters 
    ----------
    in_list_sort = list with 2 columns. First is isotope names, second is their relative error in percentage 
    in_list_copy = list with 2 columns. First is isotope names, second is their relative error in percentage 
    
    Returns 
    -------
    in_list_copy with its list rearranged to follow the first column in in_list_sort  
    
    """
    sorted_sort = sorted(in_list_sort.items(),key=operator.itemgetter(1))
    sorted_copy = sorted(in_list_copy.items(),key=operator.itemgetter(1))
    values_copy_sorted = []
    length1 = len(sorted_sort)
    length2 = len(sorted_copy)
    for x in range(0,length1):
        keys_sort = sorted_sort[x][0]
        for y in range(0,length2): 
            keys_copy = sorted_copy[y][0]
            if keys_sort == keys_copy: 
                values_copy_sorted.append(sorted_copy[y])
                break
    return sorted_sort,values_copy_sorted

In [ ]:
sorted_relative_error_isotope_51_9, sorted_relative_error_isotope_33_9 = sort_relative_error(relative_error_isotope_51_9,relative_error_isotope_33_9)
sorted_relative_error_isotope_51_6, sorted_relative_error_isotope_33_6 = sort_relative_error(relative_error_isotope_51_6,relative_error_isotope_33_6)

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
plt.axhline(y=total_relative_error_6,color='grey',label='Relative error for total spent fuel mass')
ax.scatter(*zip(*sorted_relative_error_isotope_51_6),color = 'indianred', label='Relative error for each isotope, CYCLUS Data Burn up = 51 GWD/tHM', marker = "o")
ax.scatter(*zip(*sorted_relative_error_isotope_33_6),color = 'steelblue', label='Relative error for each isotope, CYCLUS Data Burn up = 51 GWD/tHM', marker = "o")
box = ax.get_position()
ax.set_position([box.x0,box.y0 + box.height,box.width,box.height*1])
handles, labels = ax.get_legend_handles_labels()
plt.xticks(rotation=90)
ax.set_ylim(bottom=-100,top=100)
ax.legend(handles, labels, fontsize=13,loc='upper center',bbox_to_anchor=(0.7,1.25),fancybox=True)
ax.set_ylabel('Relative Error (%)', fontsize=18)
ax.set_title('Relative Error between spent fuel mass from UDB data and CYCLUS data for each isotope in 2000', fontsize=17)
plt.savefig('figures/relative_error_2000.png', bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
plt.axhline(y=total_relative_error_9,color='grey',label='Relative error for total spent fuel mass')
ax.scatter(*zip(*sorted_relative_error_isotope_51_9),color = 'indianred', label='Relative error for each isotope, CYCLUS Data Burn up = 51 GWD/tHM', marker = "o")
ax.scatter(*zip(*sorted_relative_error_isotope_33_9),color = 'steelblue', label='Relative error for each isotope, CYCLUS Data Burn up = 51 GWD/tHM', marker = "o")
box = ax.get_position()
ax.set_position([box.x0,box.y0 + box.height,box.width,box.height*1])
handles, labels = ax.get_legend_handles_labels()
plt.xticks(rotation=90)
ax.set_ylim(bottom=-100,top=100)
ax.legend(handles, labels, fontsize=13,loc='upper center',bbox_to_anchor=(0.7,1.25),fancybox=True)
ax.set_ylabel('Relative Error (%)', fontsize=18)
ax.set_title('Relative Error between spent fuel mass from UDB data and CYCLUS data for each isotope in 2014', fontsize=17)
plt.savefig('figures/relative_error_2014.png', bbox_inches="tight")

** CURIE and CYCLUS data on same plot for total spent fuel mass** 

In [ ]:
index = spent_fuel_mass_cum.index.values
index2 = ['1975-01-01','1980-01-01','1985-01-01','1990-01-01','1995-01-01','2000-01-01','2005-01-01','2010-01-01','2013-01-01']
data = spent_fuel_mass_cum.values
data2 = total_spentfuel_array_cum
dts = pd.to_datetime(index)
dts2 = pd.to_datetime(index2)

fig, ax = plt.subplots(figsize=(15,7))
ax.plot(dts, data,color = 'steelblue', label='UDB data', marker = ".")
ax.scatter(dts2, data2, color = 'indianred', label='CYCLUS data Decay On', marker = "o")
ax.grid()
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, fontsize=18)
ax.xaxis.set_major_locator(mdates.YearLocator(5))
ax.xaxis.set_major_formatter(mdates.DateFormatter('\n\n%Y'))
ax.set_xlabel('Discharge Year ', fontsize=18)
ax.set_ylabel('Total mass of spent fuel (MTHM)', fontsize=18)
ax.set_title('Cumulative mass of spent fuel vs. discharge year', fontsize=22)
plt.savefig('figures/cumulative_mass_spent_fuel_decayon.png', dpi=300)